# Jupyter Notebook Basics

**Cell modes**: There are different cell modes, the most important ones are:

* Code
* Markdown

This cell is written with Markdown. Double-click it to see the source.

**Edit vs. Command Mode**

* Edit mode: click into a cell or hit `Enter`
* Command mode: hit ESC

**Keyboard shortcuts (command mode)**

* `Shift+Enter`: Run the cell
* `↑ (up-arrow)` and `↓ (down-arrow)`: Move cell selector up or down
* `b` and `a`: Insert a new cell *below* or *above*
* `dd`: Delete the current cell
* `m`: Switch cell type to Markdown
* `y`: Switch cell type to code

# Python Basics

### Get access to additional functionality by *importing* packages

In [ ]:
import math
math.sqrt(49)

### Everything is an object. Objects have attributes and methods.

In [ ]:
import datetime as dt
timestamp = dt.datetime(2020, 1, 31, 14, 30) # timestamp is a datetime object
timestamp

In [ ]:
timestamp.year  # attribute (no brackets)

In [ ]:
timestamp.strftime('%Y-%m-%d')  # method (brackets) "string format time"

### Lists and Dictionaries are the most important data structures

In [ ]:
# List
file_names = ["one.xlsx", "two.xlsx", "three.xlsx", "four.xlsx"]

In [ ]:
# Dictionary
exchange_rates = {"EURUSD": 1.1152, "GBPUSD": 1.2454, "AUDUSD": 0.6161}

### Indexing and Slicing

In [ ]:
# Python is zero-based (unlike Excel)
file_names[0]

In [ ]:
file_names[1:3]

In [ ]:
file_names[1:]

In [ ]:
file_names[:]

### Accessing Dictionary elements

In [ ]:
exchange_rates['EURUSD']

### For loops

In [ ]:
for file_name in file_names:
    print(file_name)

### Functions

In [ ]:
def fahrenheit_to_celsius(degrees):
    return (degrees-32) * (5/9)

In [ ]:
fahrenheit_to_celsius(100)

# Pandas Basics

In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

### A DataFrame has an Index, Columns, and Data

In [ ]:
# Index ~ observations, Columns ~ variables
df = pd.DataFrame(data={"NYC": [300.1, 100.2, 123.45],
                        "Chicago": [400.3, 300.4, 334.2],
                        "San Francisco": [1000.5, 1100.6, 993.4]})

### GOTCHA: methods return copies!

In [ ]:
df.sort_values('NYC')

In [ ]:
# df is unchanged
df

In [ ]:
# If you want to change the original DataFrame, do this:
df = df.sort_values('NYC')
df

### Data Selection

In [ ]:
# Label-based data selection
df.loc[:, ['Chicago']]

In [ ]:
df.loc[[0, 1], ['Chicago', 'NYC']]

In [ ]:
# Integer-based data selection
df.iloc[0, 0]

In [ ]:
df.iloc[:2, :2]

### GOTCHA: Series (1d) vs DataFrames (2d)

In [ ]:
# This is a pandas Series (1d)
# Series have many things in common with 2d DataFrames, but
# they don't do data alignment (we'll see further down what that is)
df.loc[:, 'Chicago']

### Column selection shortcut

In [ ]:
# df.loc[:, 'Chicago'] is the same as:
df['Chicago']

### Vectorization and Data Alignment

In [ ]:
# elementwise without loops ("vectorization")
df

In [ ]:
df * 10

In [ ]:
df['NYC'] + df['Chicago']

In [ ]:
df2 = pd.DataFrame(data={"San Francisco": [10.1, 10.2, 12.4],
                        "NYC": [11.1, 10.2, 12.45],
                        "Chicago": [40.3, 30.4, 33.2]})
df2

In [ ]:
df

In [ ]:
df + df2

# Time Series Analysis with pandas and Plotly

In [ ]:
# Read a sheet of an Excel file
msft = pd.read_excel("xlsx/MSFT.xlsx")
msft

In [ ]:
msft.info()

In [ ]:
msft.describe()

In [ ]:
msft = msft.sort_index()

In [ ]:
msft = msft.set_index('Date')
msft

### Powerful selection based on date strings

In [ ]:
msft.loc["2019", "Adj Close"]

In [ ]:
msft.loc["2019-06":"2020-05", "Adj Close"].plot()

## Shifting and Percentage Changes

In [ ]:
msft_close = msft.loc[:, ["Adj Close"]].copy()

In [ ]:
msft_close.head()

In [ ]:
msft_close.shift(1).head()

In [ ]:
returns = np.log(msft_close / msft_close.shift(1))
returns = returns.rename(columns={"Adj Close": "returns"})
returns.head()

In [ ]:
# Plot a histogram with the daily log returns
returns.plot.hist()

In [ ]:
# Simple Returns
simple_rets = msft_close.pct_change()
simple_rets = simple_rets.rename(columns={"Adj Close": "simple rets"})
simple_rets.head()

# Rebasing and Correlation

In [ ]:
parts = []  # List to collect individual DataFrames
for ticker in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    # "usecols" allows us to only read in the Date and Adj Close
    adj_close = pd.read_excel(f"xlsx/{ticker}.xlsx",
                            index_col="Date", usecols=["Date", "Adj Close"])
    # Rename the column into the ticker symbol
    adj_close = adj_close.rename(columns={"Adj Close": ticker})
    # Append the stock's DataFrame to the parts list
    parts.append(adj_close)

In [ ]:
# Data alignment at work: Combine the 4 DataFrames into a single DataFrame
adj_close = pd.concat(parts, axis=1)
adj_close

In [ ]:
# Only use rows where we have data for all stocks
adj_close = adj_close.dropna()
adj_close.info()

### Let's write a DataFrame to a new Excel file

In [ ]:
adj_close.to_excel('adj_close.xlsx')

In [ ]:
# Use a sample from June 2019 - May 2020
adj_close_sample = adj_close.loc["2019-06":"2020-05", :]
rebased_prices = adj_close_sample / adj_close_sample.iloc[0, :] * 100
rebased_prices.head(2)

In [ ]:
rebased_prices.plot()

In [ ]:
# Correlation of daily log returns
returns = np.log(adj_close / adj_close.shift(1))
returns.corr()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.imshow(returns.corr(),
                x=adj_close.columns,
                y=adj_close.columns,
                color_continuous_scale=list(
                    reversed(px.colors.sequential.RdBu)),
                zmin=-1, zmax=1)
fig.show()

## Resampling

In [ ]:
# Downsampling
end_of_month = adj_close.resample("M").last()
end_of_month.head()

In [ ]:
# Upsampling
end_of_month.resample("D").asfreq().head()  # No transformation

In [ ]:
end_of_month.resample("W-FRI").ffill().head()  # Forward fill

## Rolling Windows

In [ ]:
# Plot the moving average for MSFT with data from 2019
msft19 = msft.loc["2019", ["Adj Close"]].copy()
# Add the 25 day moving average as a new column to the DataFrame
msft19.loc[:, "25day average"] = msft19["Adj Close"].rolling(25).mean()
msft19.plot()

## If you can, avoid Excel files as data source

`%%time` is a *cell magic* that prints the time spent in the cell  
Use `%%timeit` to get the average of multiple runs

In [ ]:
%%time
parts = []  # List to collect individual DataFrames
for ticker in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    # "usecols" allows us to only read in the Date and Adj Close
    adj_close = pd.read_excel(f"xlsx/{ticker}.xlsx",
                            index_col="Date", parse_dates=["Date"],
                            usecols=["Date", "Adj Close"])
    # Rename the column into the ticker symbol
    adj_close = adj_close.rename(columns={"Adj Close": ticker})
    # Append the stock's DataFrame to the parts list
    parts.append(adj_close)

In [ ]:
%%time
parts = []  # List to collect individual DataFrames
for ticker in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    # "usecols" allows us to only read in the Date and Adj Close
    adj_close = pd.read_csv(f"csv/{ticker}.csv",
                            index_col="Date", parse_dates=["Date"],
                            usecols=["Date", "Adj Close"])
    # Rename the column into the ticker symbol
    adj_close = adj_close.rename(columns={"Adj Close": ticker})
    # Append the stock's DataFrame to the parts list
    parts.append(adj_close)